In [1]:
import dagger as dag
from helper import load_raster
import matplotlib.pyplot as plt
import numpy as np

# The following line is a jupyter lab command telling it to use the interactive widget ipympl for matplotlib
%matplotlib widget

# dem = load_raster("example.tif")
dem = load_raster("../../../../FastFlood2.0/FastFlood2_Boris/data/DEM_1m_crop.tif")

con = dag.D8N(dem["nx"], dem["ny"], dem["dx"], dem["dy"], dem["x_min"], dem["y_min"])
gf = dag.graph(dem["nx"] * dem["ny"], 8)
gf.init_graph(con)
gf.set_LMR_method(dag.LMR.cordonnier_fill)
PPdem = gf.compute_graph(dem['array'].ravel(), con, True, True)
HS = dag.hillshade(con,PPdem)
rshp = [dem['ny'], dem['nx']]
rshp

SDA = gf.accumulate_constant_downstream_SFD(con,dem['dx'] * dem['dy'])

gradient = gf.get_links_gradient(con, PPdem)

In [ ]:
weights = gf.get_link_weights(gradient,0)
MFA_0 = gf.accumulate_constant_downstream_MFD(con,weights,1)

In [ ]:
fig, ax = plt.subplots()
cb = ax.imshow(np.log10(MFA_0).reshape(rshp), cmap = 'Blues')
ax.imshow(HS.reshape(rshp), cmap = 'gray', alpha = 0.1)
plt.colorbar(cb, label = "Drainage area")
ax.grid(ls = '--', alpha = 0.2, color = 'k')

This is the most divergent scenario and may be rarely relevant for drainage area but it illustrates well the differences of pattern with single flow direction. If you zoom on the river with the square zoom option (only feasible with %matplotlib widget active, you'll notice they are multiple pixel wide (this is a low0res DEM, it would be more obvious on a lidar but bigger to distribute as test site). The hillslopes in the eastern part of thhis landscape are also a good exemple: they are very linear and you see that the flow struggle to converge and form channels.

OK now let's consider the partition to be function of the squareroot of the slope, this would be in accordance with mannings equations for example.


In [ ]:
weights = gf.get_link_weights(gradient,0.5)
MFA_05 = gf.accumulate_constant_downstream_MFD(con,weights,900)

In [ ]:
fig, ax = plt.subplots()
cb = ax.imshow(np.log10(MFA_05).reshape(rshp), cmap = 'Blues')
ax.imshow(HS.reshape(rshp), cmap = 'gray', alpha = 0.1)
plt.colorbar(cb, label = "Drainage area")
ax.grid(ls = '--', alpha = 0.2, color = 'k')

And it leads to more channelised flow.
Now let's compare multiple values with a small animation:
First I create a figure holder and in the cell after I'll make a loop updating it

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.log10(MFA_0).reshape(rshp), cmap = 'Blues')
ax.imshow(HS.reshape(rshp), cmap = 'gray', alpha = 0.1)
plt.colorbar(im, label = "Drainage area")
ax.grid(ls = '--', alpha = 0.2, color = 'k')

In [ ]:
import time
p = 0
for i in range(1000):
    dp = 0.05
    if(p>1):
        dp = 0.2
    p += dp
    if(p > 15):
        p = 0
    weights = gf.get_link_weights(gradient,p)
    MFA = gf.accumulate_constant_downstream_MFD(con,weights,900)
    im.set_data(np.log10(MFA).reshape(rshp))
    yo = ax.set_title("p = %s"%(round(p,2)))
    fig.canvas.draw()
    

In [ ]:
%timeit PPdem = gf.compute_graph("cordonnier_carve", dem['array'].ravel(), con, False, False)

In [ ]:
%timeit PPdem = gf.compute_graph("cordonnier_carve", dem['array'].ravel(), con, False, True)

In [ ]:

%timeit PPdem = gf.compute_graph("cordonnier_carve", dem['array'].ravel(), con, True, True)